In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Enhanced Optimization for Radar and Communications Applications 


The Low Autocorrelation Binary Sequences (LABS) is an important and challenging optimization problem with applications related to radar, telecommunications, and other signal related applications. This CUDA-Q Academic module will focus on a clever quantum-enhanced hybrid method developed in a collaboration between Kipu Quantum, University of the Basque Country EHU, and NVIDIA for solving the LABS problem. (This notebook was jointly developed with input from all collaborators.)

Other CUDA-Q Academic modules like [Divide and Conquer MaxCut QAOA](https://github.com/NVIDIA/cuda-q-academic/tree/main/qaoa-for-max-cut) and [Quantum Finance](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb), demonstrate how quantum computing can be used outright to solve optimization problems. This notebook demonstrates a slightly different approach. Rather than considering QPUs as the tool to produce the final answer, it demonstrates how quantum can be used to enhance the effectiveness of leading classical methods.  

The benefits of such an approach were highlighted in [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1).  This notebook, co-created with the authors of the paper, will allow you to explore the findings of their research and write your own CUDA-Q code that builds a representative quantum-enhanced workflow for solving the LABS problem. Moreover, it will introduce advancements in counteradiabatic optimization techniques on which reduce the quantum resources required to run on a QPU.

**Prerequisites:** This lab assumes you have a basic knowledge of quantum computing, including operators, gates, etc.  For a refresher on some of these topics, explore the [Quick start to Quantum](https://github.com/NVIDIA/cuda-q-academic/tree/main/quick-start-to-quantum) series.

**In this lab you will:**
* 1. Understand the LABS problem and its relation ot radar and communication applications.
* 2. Solve LABS classically with memetic tabu search and learn about the limitations of such methods.
* 3. Code a couteradiabatic algorithm using CUDA-Q to produce approximate solutions to the LABS problem.
* 4. Use the CUDA-Q results to seed your tabu search and understand the potential benefits of this approach.


**Terminology you will use:**
* Low autocorrelation of binary sequences (LABS)
* counteradiabatic optimization
* memetic-tabu search

**CUDA-Q Syntax you will use:**
* cudaq.sample()
* @cudaq.kernel
* ry(), rx(), rz(), x(), h() 
* x.ctrl()

Run the code below to initialize the libraries you will need.

In [ ]:
import cudaq
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils

## The LABS problem and applications

The **Low Autocorrelation Binary Sequences (LABS)** problem is fundamental to many applications, but originated with applications to radar. 

Consider a radar that monitors airport traffic.  The radar signal sent to detect incoming planes must have as much range as possible to ensure safe approaches are planned well in advance.  The range of a radar signal can be increased by sending a longer pulse.  However, in order to differentiate between multiple objects, pulses need to be short to provide high resolution. So, how do you handle situations where you need both?

One solution is a technique called pulse compression.  The idea is to send a long signal, but vary the phase at regular intervals such that the resolution is increased. Generally, the initial signal will encode a binary sequence of phase shifts, where each interval corresponds to a signal with a 0 or 180 degree phase shift. 

The tricky part is selecting an optimal encoding sequence.  When the signal returns, it is fed into a matched filter with the hope that a singular sharp peak will appear, indicating clear detection.  The autocorrelation of the original signal, or how similar the signal is to itself,  determines if a single peak or a messier signal with sidelobes will be detected. A signal should have high autocorrelation when overlayed on top of itself, but low autocorrelation when shifted with a lag. 

Consider the image below.  The signal on the left has a crisp single peak while the single on the right produces many undesirable sidelobes which can inhibit clear detection.  

<img src="images/quantum_enhanced_optimization_LABS/radar.png" width="800">


So, how do you select a good signal?   This is where LABS comes in, defining these questions as a binary optimization problem. Given a binary sequence of length $N$, $(s_1 \cdots s_N) \in {\pm 1}^N$, the goal is to minimize the following objective function.

$$ E(s) = \sum_{k=1}^{N-1} C_k^2 $$

Where $C_k$ is defined as. 

 $$C_k= \sum_{i=1}^{N-k} s_is_{i+k}$$


So, each $C_k$ computes how similar the original signal is to the shifted one for each offset value $k$.  To explore this more, try the interactive widget linked [here](https://nvidia.github.io/cuda-q-academic/interactive_widgets/labs_visualization.html).  See if you can select a very good and very poor sequence and see the difference for the case of $N=7$.

## Classical Solution of the LABS problem

The LABS problem is tricky to solve for a few reasons. First, the configuration space grows exponentially.  Second, underlying symmetries of the problem result in many degeneracies in the optimization landscape severely inhibiting local search methods. 

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 1:</h3>
    <p style="font-size: 16px; color: #333;">
Using the widget above, try to find some of the symmetries for the LABS problem. That is, for a fixed bitstring length, can you find patterns to produce the same energy with different pulse patterns. 
</div>

The best known performance for a classical optimization technique is Memetic Tabu search (MTS) which exhibits a scaling of $O(1.34^N)$.  The MTS algorithm is depicted below.  It begins with a randomly selected population of bitstrings and finds the best solution from them.  Then, a child is selected by sampling directly from or combining multiple bitstrings from the population.  The child is mutated with probability $p_{mutate}$ and then input to a tabu search, which performs a modified greedy local search starting from the child bitstring.  If the result is better than the best in the population, it is updated as the new leader and randomly replaces a  bitstring in the population.


<img src="images/quantum_enhanced_optimization_LABS/mts_algorithm.png" width="500">

Such an approach is fast, parallelizable, and allows for exploration with improved searching of the solution landscape.  

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 2:</h3>
    <p style="font-size: 16px; color: #333;">
Before exploring any quantum approach, get a sense for how MTS works by coding it yourself based generally on the figure above. Algorithms for the combine and mutate steps are provided below as used in the paper. You may need to research more specific details of the process, especially for how tabu search is performed. The MTS procedure should output and optimal bitstring and its energy.  Also, write a function to visualize the results including the energy distribution of the final population.
</div>



<img src="images/quantum_enhanced_optimization_LABS/combine_mutate.png" width="400">



## Exercise 1: LABS Problem Definition and Symmetries

We study the **Low Autocorrelation Binary Sequence (LABS)** problem, where the goal
is to find a binary sequence
\(\mathbf{s} = (s_1, \dots, s_N)\) with \(s_i \in \{-1, +1\}\) that minimizes
aperiodic autocorrelation across all non-zero lags.

The LABS energy is defined as:

$$
E(\mathbf{s}) = \sum_{k=1}^{N-1} C_k(\mathbf{s})^2,
\quad
C_k(\mathbf{s}) = \sum_{i=1}^{N-k} s_i s_{i+k}.
$$

This objective exhibits several **exact symmetries**, including:
- Global sign flip: \(\mathbf{s} \mapsto -\mathbf{s}\)
- Sequence reversal
- Reversal combined with sign flip

These invariances imply that many distinct sequences share the same energy.
We exploit these properties as **self-validation checks** for our
`compute_energy` implementation.


## LABS Problem Formulation

A **Low Autocorrelation Binary Sequence (LABS)** of length \(N\) is a sequence
\(\mathbf{s} = (s_1, s_2, \ldots, s_N)\) where each element is binary in the
\(\pm 1\) representation:

$$
s_i \in \{-1, +1\}, \quad i = 1, \ldots, N.
$$

For each lag \(k = 1, 2, \ldots, N-1\), define the (aperiodic) autocorrelation:

$$
C_k(\mathbf{s}) = \sum_{i=1}^{N-k} s_i s_{i+k}.
$$

The LABS objective (energy) is:

$$
E(\mathbf{s}) = \sum_{k=1}^{N-1} \left(C_k(\mathbf{s})\right)^2.
$$

The optimization problem is therefore:

$$
\mathbf{s}^\star
= \arg\min_{\mathbf{s} \in \{-1,+1\}^N} E(\mathbf{s}).
$$

Lower energy corresponds to lower autocorrelation across all lags, meaning the
sequence is less predictable under shifts.


## Exercise 2: Classical Memetic Tabu Search (MTS)

Before introducing any quantum enhancement, we implement a **classical baseline**
based on Memetic Tabu Search (MTS), following the reference paper.

MTS combines:
- A population-based genetic framework
- Local refinement via tabu search
- Controlled mutation to maintain diversity

Each generation consists of:
1. Selecting parent sequences from the population
2. Recombining parents via one-point crossover
3. Applying random bit flips with probability \(p_{\text{mut}}\)
4. Refining the child using tabu-guided local search
5. Updating the population if improvement is found

The algorithm returns the best sequence found and its corresponding LABS energy.
This classical baseline is later used for both validation and benchmarking.


In [ ]:
#TODO - Write code to perform MTS
"""
Memetic Tabu Search (MTS) for the LABS Problem
===============================================
Implements the MTS algorithm as described in the paper:
"Scaling advantage with quantum-enhanced memetic tabu search for LABS"
"""

import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple


# ============================================================================
# LABS Energy Functions
# ============================================================================

def compute_Ck(s: np.ndarray, k: int) -> int:
    """
    Compute the k-th autocorrelation coefficient C_k.
    C_k = sum_{i=1}^{N-k} s_i * s_{i+k}
    
    Args:
        s: Binary sequence of +1/-1 values
        k: Lag offset (1 to N-1)
    
    Returns:
        Autocorrelation value C_k
    """
    N = len(s)
    return int(np.sum(s[:N-k] * s[k:]))


def compute_energy(s: np.ndarray) -> int:
    """
    Compute the LABS energy E(s) = sum_{k=1}^{N-1} C_k^2
    
    Args:
        s: Binary sequence of +1/-1 values
    
    Returns:
        Energy value (lower is better)
    """
    N = len(s)
    energy = 0
    for k in range(1, N):
        Ck = compute_Ck(s, k)
        energy += Ck * Ck
    return energy


# ============================================================================
# MTS Helper Functions: Combine and Mutate
# ============================================================================

def combine(p1: np.ndarray, p2: np.ndarray) -> np.ndarray:
    """
    Combine two parent sequences using single-point crossover.
    Algorithm 3, Line 1-3 from the paper.
    
    Args:
        p1: First parent sequence
        p2: Second parent sequence
    
    Returns:
        Child sequence
    """
    N = len(p1)
    # Choose cut point k uniformly from {1, ..., N-1}
    k = np.random.randint(1, N)
    # Return p1[0:k] || p2[k:N]
    child = np.concatenate([p1[:k], p2[k:]])
    return child


def mutate(s: np.ndarray, p_mut: float) -> np.ndarray:
    """
    Mutate a sequence by flipping each bit with probability p_mut.
    Algorithm 3, Line 4-8 from the paper.
    
    Args:
        s: Input sequence
        p_mut: Mutation probability per bit
    
    Returns:
        Mutated sequence
    """
    s = s.copy()
    for i in range(len(s)):
        if np.random.random() < p_mut:
            s[i] = -s[i]  # Flip: +1 -> -1 or -1 -> +1
    return s


# ============================================================================
# Tabu Search Implementation
# ============================================================================

def tabu_search(s: np.ndarray, max_iter: int = 100, tabu_tenure: int = 7) -> Tuple[np.ndarray, int]:
    """
    Perform Tabu Search starting from sequence s.
    
    Tabu Search is a local search that:
    1. At each step, evaluates all single-bit-flip neighbors
    2. Selects the best non-tabu move (or best aspiration move if it beats global best)
    3. Adds the flipped position to tabu list to avoid cycling
    
    Args:
        s: Starting sequence
        max_iter: Maximum iterations
        tabu_tenure: How long a move stays in tabu list
    
    Returns:
        Tuple of (best_sequence, best_energy)
    """
    N = len(s)
    current = s.copy()
    current_energy = compute_energy(current)
    
    best = current.copy()
    best_energy = current_energy
    
    # Tabu list: stores the iteration when each position becomes non-tabu
    tabu_list = np.zeros(N, dtype=int)
    
    for iteration in range(max_iter):
        best_neighbor_energy = float('inf')
        best_flip_pos = -1
        
        # Evaluate all single-bit-flip neighbors
        for i in range(N):
            # Create neighbor by flipping bit i
            neighbor = current.copy()
            neighbor[i] = -neighbor[i]
            neighbor_energy = compute_energy(neighbor)
            
            # Check if move is tabu
            is_tabu = tabu_list[i] > iteration
            
            # Aspiration criterion: accept if it beats global best
            aspiration = neighbor_energy < best_energy
            
            if (not is_tabu or aspiration) and neighbor_energy < best_neighbor_energy:
                best_neighbor_energy = neighbor_energy
                best_flip_pos = i
        
        # If no improving move found, break
        if best_flip_pos == -1:
            break
        
        # Make the move
        current[best_flip_pos] = -current[best_flip_pos]
        current_energy = best_neighbor_energy
        
        # Add to tabu list
        tabu_list[best_flip_pos] = iteration + tabu_tenure
        
        # Update global best
        if current_energy < best_energy:
            best = current.copy()
            best_energy = current_energy
    
    return best, best_energy


# ============================================================================
# Main MTS Algorithm
# ============================================================================

def random_sequence(N: int) -> np.ndarray:
    """Generate a random binary sequence of +1/-1."""
    return np.random.choice([-1, 1], size=N)


def mts(N: int, 
        population_size: int = 20, 
        max_generations: int = 100,
        p_mut: float = 0.05,
        tabu_iter: int = 100,
        tabu_tenure: int = 7,
        initial_population: List[np.ndarray] = None,
        verbose: bool = True) -> Tuple[np.ndarray, int, List[np.ndarray], List[int]]:
    """
    Memetic Tabu Search for the LABS problem.
    
    Algorithm:
    1. Initialize population (random or provided)
    2. Find best solution in population
    3. For each generation:
       a. Select two parents from population
       b. Combine parents to create child
       c. Mutate child with probability p_mut
       d. Apply Tabu Search to child
       e. If result improves best, update and replace a random population member
    
    Args:
        N: Sequence length
        population_size: Number of sequences in population
        max_generations: Maximum number of generations
        p_mut: Mutation probability
        tabu_iter: Max iterations for tabu search
        tabu_tenure: Tabu tenure length
        initial_population: Optional initial population (for quantum seeding)
        verbose: Print progress
    
    Returns:
        Tuple of (best_sequence, best_energy, final_population, energy_history)
    """
    # Initialize population
    if initial_population is not None:
        population = [seq.copy() for seq in initial_population[:population_size]]
        # Fill remaining if needed
        while len(population) < population_size:
            population.append(random_sequence(N))
    else:
        population = [random_sequence(N) for _ in range(population_size)]
    
    # Compute initial energies
    energies = [compute_energy(s) for s in population]
    
    # Find initial best
    best_idx = np.argmin(energies)
    best = population[best_idx].copy()
    best_energy = energies[best_idx]
    
    energy_history = [best_energy]
    
    if verbose:
        print(f"Initial best energy: {best_energy}")
    
    # Main loop
    for gen in range(max_generations):
        # Select two parents (tournament selection)
        idx1, idx2 = np.random.choice(population_size, 2, replace=False)
        p1, p2 = population[idx1], population[idx2]
        
        # Combine
        child = combine(p1, p2)
        
        # Mutate
        child = mutate(child, p_mut)
        
        # Tabu Search
        child, child_energy = tabu_search(child, max_iter=tabu_iter, tabu_tenure=tabu_tenure)
        
        # Update population if child is better than worst
        worst_idx = np.argmax(energies)
        if child_energy < energies[worst_idx]:
            population[worst_idx] = child
            energies[worst_idx] = child_energy
        
        # Update global best
        if child_energy < best_energy:
            best = child.copy()
            best_energy = child_energy
            if verbose:
                print(f"Gen {gen+1}: New best energy = {best_energy}")
        
        energy_history.append(best_energy)
    
    if verbose:
        print(f"\nFinal best energy: {best_energy}")
        print(f"Best sequence: {best}")
    
    return best, best_energy, population, energy_history


# ============================================================================
# Visualization Functions
# ============================================================================

def visualize_results(population: List[np.ndarray], 
                      energy_history: List[int],
                      best_sequence: np.ndarray,
                      best_energy: int,
                      title: str = "MTS Results") -> plt.Figure:
    """
    Visualize MTS results including energy distribution and convergence.
    
    Args:
        population: Final population
        energy_history: Best energy at each generation
        best_sequence: Best sequence found
        best_energy: Best energy found
        title: Plot title
    
    Returns:
        matplotlib Figure object
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. Energy convergence over generations
    ax1 = axes[0, 0]
    ax1.plot(energy_history, 'b-', linewidth=2)
    ax1.set_xlabel('Generation', fontsize=12)
    ax1.set_ylabel('Best Energy', fontsize=12)
    ax1.set_title('Energy Convergence', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # 2. Final population energy distribution
    ax2 = axes[0, 1]
    final_energies = [compute_energy(s) for s in population]
    ax2.hist(final_energies, bins=20, color='green', alpha=0.7, edgecolor='black')
    ax2.axvline(x=best_energy, color='red', linestyle='--', linewidth=2, label=f'Best: {best_energy}')
    ax2.set_xlabel('Energy', fontsize=12)
    ax2.set_ylabel('Count', fontsize=12)
    ax2.set_title('Final Population Energy Distribution', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Best sequence visualization
    ax3 = axes[1, 0]
    N = len(best_sequence)
    colors = ['blue' if s == 1 else 'red' for s in best_sequence]
    ax3.bar(range(N), best_sequence, color=colors, edgecolor='black')
    ax3.set_xlabel('Position', fontsize=12)
    ax3.set_ylabel('Value (+1/-1)', fontsize=12)
    ax3.set_title(f'Best Sequence (Energy={best_energy})', fontsize=14)
    ax3.set_ylim(-1.5, 1.5)
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    # 4. Autocorrelation of best sequence
    ax4 = axes[1, 1]
    Ck_values = [compute_Ck(best_sequence, k) for k in range(1, N)]
    ax4.bar(range(1, N), Ck_values, color='purple', alpha=0.7, edgecolor='black')
    ax4.set_xlabel('Lag k', fontsize=12)
    ax4.set_ylabel('C_k', fontsize=12)
    ax4.set_title('Autocorrelation Coefficients', fontsize=14)
    ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax4.grid(True, alpha=0.3)
    
    fig.suptitle(title, fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    return fig


def sequence_to_string(s: np.ndarray) -> str:
    """Convert +1/-1 sequence to '+'/'-' string."""
    return ''.join(['+' if x == 1 else '-' for x in s])


# ============================================================================
# Example Usage
# ============================================================================

if __name__ == "__main__":
    # Run MTS on a small example
    N = 20
    print(f"Running MTS for LABS with N={N}")
    print("=" * 50)
    
    best_seq, best_e, final_pop, history = mts(
        N=N,
        population_size=20,
        max_generations=50,
        p_mut=0.05,
        tabu_iter=100,
        verbose=True
    )
    
    print(f"\nBest sequence as string: {sequence_to_string(best_seq)}")
    
    # Visualize
    fig = visualize_results(final_pop, history, best_seq, best_e, f"MTS Results for N={N}")
    plt.savefig('mts_results.png', dpi=150, bbox_inches='tight')
    plt.show()

### Optional: GPU-Accelerated Brute-Force Validation (CuPy)

To validate our heuristic results beyond CPU-feasible brute force, we implemented an **optional
GPU-accelerated exhaustive search** using CuPy.

This approach enumerates all \(2^N\) binary sequences in GPU batches, computes the LABS energy
fully vectorized across the batch, and tracks the global minimum energy.

This implementation is intended **strictly as a ground-truth oracle** for moderate values of \(N\)
when GPU resources allow. For larger \(N\), exhaustive search becomes computationally prohibitive,
and we rely on heuristic optimization methods (MTS and quantum-seeded MTS).

We use this brute-force solution to:
- Verify correctness of the `compute_energy` implementation
- Validate convergence of the classical MTS baseline
- Provide a trusted reference point for later quantum-enhanced methods


In [ ]:
import time
import numpy as np
import cupy as cp

# ==========================================
# 1. 核心能量計算 (GPU Vectorized)
# ==========================================
def calculate_batch_energy(gpu_sequences_matrix):
    """
    輸入: shape (Batch_Size, N) 的 GPU 矩陣 (值為 +1/-1)
    輸出: shape (Batch_Size,) 的能量陣列
    """
    batch_size, N = gpu_sequences_matrix.shape
    energies = cp.zeros(batch_size, dtype=cp.float32)
    
    # 利用矩陣切片並行計算所有 k 的自相關
    # 這是 LABS 能量公式 E = sum(Ck^2) 的向量化實作
    for k in range(1, N):
        # 這裡不使用迴圈，而是矩陣位移相乘
        # term = s[i] * s[i+k]
        term = gpu_sequences_matrix[:, :N-k] * gpu_sequences_matrix[:, k:]
        
        # 對每一列求和得到 Ck，然後平方
        Ck = cp.sum(term, axis=1)
        energies += Ck**2
        
    return energies

# ==========================================
# 2. GPU 窮舉搜尋主程式
# ==========================================
def solve_exact_labs_gpu(N, batch_size=10_000_000):
    total_combinations = 1 << N
    print(f"[{time.strftime('%H:%M:%S')}] Starting GPU Exhaustive Search for N={N}...")
    print(f"   Total combinations: {total_combinations:,}")
    
    start_time = time.time()
    global_min_energy = float('inf')
    global_best_sequences = [] 
    
    for start_idx in range(0, total_combinations, batch_size):
        end_idx = min(start_idx + batch_size, total_combinations)
        
        # 1. 生成序號 (uint64)
        indices = cp.arange(start_idx, end_idx, dtype=cp.uint64)
        
        # 2. [修正點] 產生位移量: 先產生 0~N-1，再反轉變成 N-1~0
        # 這樣就避開了負數參數的問題
        shift_amounts = cp.arange(N, dtype=cp.uint64)[::-1]
        
        # 進行廣播位移 (indices shape: (Batch, 1), shift shape: (N,))
        # 結果 bits shape: (Batch, N)
        bits = (indices[:, None] >> shift_amounts) & 1
        
        # 3. 轉換為 +1/-1
        sequences = cp.where(bits == 0, 1, -1).astype(cp.int8)
        
        # 4. 計算能量
        energies = calculate_batch_energy(sequences)
        
        # 5. 找出最小值
        batch_min = float(cp.min(energies))
        
        if batch_min < global_min_energy:
            global_min_energy = batch_min
            min_mask = (energies == batch_min)
            best_seqs_gpu = sequences[min_mask]
            # 轉換回 CPU list 儲存
            global_best_sequences = [row for row in cp.asnumpy(best_seqs_gpu)]
            print(f"   > New min found: {global_min_energy} (at index {start_idx})")
            
        elif batch_min == global_min_energy:
            min_mask = (energies == batch_min)
            best_seqs_gpu = sequences[min_mask]
            new_best = cp.asnumpy(best_seqs_gpu)
            for seq in new_best:
                # 簡單去重 (如果需要)
                global_best_sequences.append(seq)

        # 釋放記憶體
        del sequences, energies, bits, indices, shift_amounts
        cp.get_default_memory_pool().free_all_blocks()

    end_time = time.time()
    elapsed = end_time - start_time
    
    print("\n" + "="*50)
    print(f"EXACT SOLUTION FOUND (N={N})")
    print(f"Minimum Energy: {global_min_energy}")
    print(f"Time Elapsed: {elapsed:.4f} seconds")
    print("="*50)
    
    return global_min_energy
# ==========================================
# 執行
# ==========================================
if _name_ == "_main_":
    # 確保使用 GPU
    try:
        import cupy
        print(f"Using GPU: {cupy.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
        
        # 執行 N=20 (約 100萬種組合，瞬間完成)
        # solve_exact_labs_gpu(20)
        
        # 挑戰 N=25 (約 3300萬種組合)
        solve_exact_labs_gpu(25)
        
        # 挑戰 N=30 (約 10億種組合，B系列 GPU 應該能在幾秒到幾分鐘內跑完)
        # solve_exact_labs_gpu(30)
        
    except ImportError:
        print("Error: CuPy not installed. Please install cupy-cuda12x to run this on GPU.")

## Building a Quantum Enhanced Workflow

Despite the effectiveness of MTS, it still exhibits exponential scaling  $O(1.34^N)$ behavior and becomes intractable for large $N$.  Quantum computing provides a potential alternative method for solving the LABS problem because the properties of entanglement, interference, and superpositon may allow for a better global search.  Recent demonstrations have even produced evidence that the quantum approximate optimization algorithm (QAOA) can be used to reduce the scaling of the LABS problem to $O(1.21^N)$ for $N$ between 28 and 40 with quantum minimum finding.

However, current quantum hardware limitations restrict solution to problems of greater than about $N=20$, meaning that it will be some time before quantum approaches can outperform the classical state of the art. It should also be noted that standard QAOA can struggle with LABS and require many layers to converge the parameters if other tricks are not employed.

The authors of [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1) cleverly explored an alternate path that combines quantum and classical approaches and might be able to provide a more near-term benefit.  Instead of expecting the quantum computer to solve the problem entirely, they asked how a quantum approach might enhance MTS.

The basic idea is that a quantum optimization routine could run first and the resulting state be sampled to produce a better population for MTS. Many such heuristics for defining the initial population are possible, but the rest of this notebook will explore their methodology, help you to build the workflow yourself, and allow you to analyze the benefits of their approach.

The first step of quantum enhanced MTS (QE-MTS) is to prepare a circuit with CUDA-Q that approximates the ground state of the Hamiltonian corresponding to the LABS problem. You could do this with any optimization algorithm such as QAOA or using an adiabatic approach.  (See the [Quantum Portfolio Optimization](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb) CUDA-Q Academic lab for a detailed comparison of these two common approaches.)

The authors of this work opted for an adiabatic approach (More on why later). Recall that the goal of an adiabatic optimization is to begin with a Hamiltonian that has an easily prepared ground state ($H_i$). Then, the adiabatic Hamiltonian $H_{ad}$ can be constructed as $H_{ad}(\lambda) = (1-\lambda)H_i +\lambda H_f $, where $\lambda$ is a function of time and $H_f$ is the Hamiltonian representing a qubit encoding of the LABS problem. 

$$H_f = 2 \sum_{i=1}^{N-2} \sigma_i^z \sum_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} \sigma_{i+k}^z 
+ 4 \sum_{i=1}^{N-3} \sigma_i^z \sum_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \sum_{k=t+1}^{N-i-t} \sigma_{i+t}^z \sigma_{i+k}^z \sigma_{i+k+t}^z$$

The authors also selected $H_i = \sum_i h^x_i \sigma^x_i $ which has an easily prepared ground state of $\ket{+}^{\otimes N}$.

The challenge for implementing the optimization procedure becomes selection of an operator that will quickly and accurately evolve to the ground state of $H_f$.  One approach is to use a so-called auxiliary countradiabatic (CD) term $H_{CD}$, which corrects diabatic transitions that jump out of the ground state during the evolution. The figure below demonstrates the benefit of using a CD correction.


<img src="images/quantum_enhanced_optimization_LABS/counteradiabatic.png" width="900">




An operator called the adiabatic gauge potential $A_{\lambda}$ is the ideal choice for the CD term as it suppresses all possible diabatic transitions, resulting in the following total system to evolve.

$$ H(\lambda) = H_{ad}(\lambda) + \lambda H_{CD} (\lambda) $$

$A(\lambda)$ is derrived from $H_{ad}(\lambda)$  (see paper for details) as it contains information about underlying physics of the problem. 

There is a problem though.  The $A(\lambda)$ term cannot be efficiently expressed exactly and needs to be approximated.  It also turns out that in the so called impulse regime, where the adiabatic evolution is very fast, $H_{cd} (\lambda)$ dominates $H_{ad}(\lambda)$, meaning that the final implementation corresponds to the operator $H(\lambda) = H^1_{cd}(\lambda)$ where  $H^1_{cd}(\lambda)$ is a first order approximation of $A(\lambda)$ (see equation 7 in the paper).

A final step is to use Trotterization to define the quantum circuit to apply $e^{-\theta (t) i H_{cd}}$. The details for this derivation are shown in the appendix of the paper. and result from equation B3 is shown below.  

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

It turns out that this implementation is more efficient than QAOA in terms of gate count. The authors calculated that for $N=67$, QAOA would require 1.4 million entangling gates while the CD approach derived here requires only 236 thousand entangling gates.


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 3:</h3>
    <p style="font-size: 16px; color: #333;">
At first glance, this equation might looks quite complicated. However, observe the structure and note two "blocks" of terms.  Can you spot them?  

They are 2 qubit terms that look like $R_{YZ}(\theta)$ or $R_{ZY}(\theta)$.

As well as 4 qubit terms that look like $R_{YZZZ}(\theta)$, $R_{ZYZZ}(\theta)$, $R_{ZZYZ}(\theta)$, or $R_{ZZZY}(\theta)$.

Thankfully the authors derive a pair of circuit implementations for the two and four qubit terms respectively, shown in the figures below.

Using CUDA-Q, write a kernel for each which will be used later to construct the full implementation.

* Hint: Remember that the adjoint of a rotation gate is the same as rotating in the opposite direction. 

* Hint: You may also want to define a CUDA-Q kernel for an R$_{ZZ}$ gate.

* Hint: Implementing a circuit from a paper is a great place where AI can help accelerate your work.  If you have access to a coding assistant, feel free to use it here.
</div>

<img src="images/quantum_enhanced_optimization_LABS/kernels.png" width="1300">


## Exercise 3: CUDA-Q Kernels for 2-Qubit and 4-Qubit Operators

### Goal

The LABS Hamiltonian contains interaction terms that map to **2-qubit** and **4-qubit Pauli products**.
To enable quantum evaluation (and later quantum-seeded / hybrid optimization), we implemented
**CUDA-Q kernels** that realize these operators as reusable circuit blocks.

---

### What we implemented

We defined three modular CUDA-Q kernels:

- **`rzz(q0, q1, θ)`**  
  A helper implementing the two-qubit interaction

  `R_ZZ(θ) = exp(-i * θ / 2 * Z ⊗ Z)`

  using the standard gate decomposition:

  `CNOT(q0, q1) → Rz(q1, θ) → CNOT(q0, q1)`.

- **`two_qubit_block(q0, q1, θ)`**  
  A 2-qubit building block that composes basis transformations with `RZZ` gates to realize the
  operator structure shown in the paper’s Fig. 3. This effectively implements coupled two-body
  interactions using single-qubit rotations and entangling gates.

- **`four_qubit_block(q0, q1, q2, q3, θ)`**  
  A 4-qubit building block following the paper’s Fig. 4 decomposition to realize higher-order
  interaction structure (Pauli-string-style terms across four qubits).

  We implemented this kernel using **explicit qubit arguments** (rather than passing a `qvector`)
  to match CUDA-Q kernel signature constraints and to keep kernel calls explicit and readable.

---

### Why this design

We intentionally kept the kernels **modular** (`RZZ → 2-qubit block → 4-qubit block`) so that we can:

- Reuse the same primitives across different LABS sizes and Hamiltonian term groupings
- Integrate these blocks into a larger **hybrid classical–quantum workflow** without rewriting circuits
- Swap decompositions or introduce approximations later if needed

This design keeps the quantum component flexible and extensible within the overall optimization pipeline.

---

### Verification

To sanity-check correctness at the **circuit-structure level**, we used `cudaq.draw` to render both blocks:

- **`test_draw_g2(θ)`** renders the 2-qubit architecture (Fig. 3)
- **`test_draw_g4(θ)`** renders the 4-qubit architecture (Fig. 4)

This confirms that our implementation matches the intended **gate ordering and entanglement pattern**
before integrating these kernels into energy evaluation or quantum-seed generation.


In [ ]:
# TODO  Write CUDA-Q kernels to apply the 2 and 4 qubit operators. 
import cudaq
import numpy as np

# ============================================================================
# RZZ Gate Helper
# ============================================================================
@cudaq.kernel
def rzz(q0: cudaq.qubit, q1: cudaq.qubit, theta: float):
    """
    Implements R_ZZ(θ) = exp(-i * θ/2 * Z ⊗ Z).
    Structure: CNOT(q0, q1) -> Rz(q1, θ) -> CNOT(q0, q1).
    """
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)

# ============================================================================
# FIG 3: Two-Qubit Block Implementation
# ============================================================================
@cudaq.kernel
def two_qubit_block(q0: cudaq.qubit, q1: cudaq.qubit, theta: float):
    """
    Implements the decomposition of R_YZ(θ)R_ZY(θ).
    Requires 2 Rzz gates and 4 single-qubit rotation gates.
    """
    # First rotation: Ryz basis transformation
    rx(np.pi/2, q0)
    rzz(q0, q1, theta)
    rx(-np.pi/2, q0)

    # Second rotation: Rzy basis transformation
    rx(np.pi/2, q1)
    rzz(q0, q1, theta)
    rx(-np.pi/2, q1)

# ============================================================================
# FIG 4: Four-Qubit Block Implementation (Fixed for qvector input)
# ============================================================================
@cudaq.kernel
def four_qubit_block(q0: cudaq.qubit, q1: cudaq.qubit, q2: cudaq.qubit, q3: cudaq.qubit, theta: float):
    """
    Implements the decomposition of R_YZZZ, R_ZYZZ, R_ZZYZ, R_ZZZY.
    This version uses individual qubit objects as arguments to match 
    the caller's signature requirements.
    """
    # --- Phase 1: Initial Transformation Layer --- 
    rx(-np.pi/2, q0)
    ry(np.pi/2, q1)
    ry(-np.pi/2, q2)

    # --- Phase 2: First Entanglement Sequence ---
    rzz(q0, q1, -np.pi/2)
    rzz(q2, q3, -np.pi/2)

    rx(np.pi/2, q0)
    ry(-np.pi/2, q1)
    ry(np.pi/2, q2)
    rx(-np.pi/2, q3)

    # --- Phase 3: Secondary Entanglement (Theta Injection) ---
    rx(-np.pi/2, q1)
    rx(-np.pi/2, q2)

    rzz(q1, q2, theta)
    
    rx(np.pi/2, q1)
    rx(np.pi, q2)

    # --- Phase 4: Long-range Entanglement ---
    ry(np.pi/2, q1)
    rzz(q0, q1, np.pi/2)
    rx(np.pi/2, q0)
    ry(-np.pi/2, q1)

    # --- Phase 5: Central Symmetry Layer ---
    # Strictly following the gate sequence with individual qubit variables
    rzz(q1, q2, -theta) 
    rx(np.pi/2, q1)
    rx(-np.pi, q2)

    rzz(q1, q2, -theta)
    rx(-np.pi, q1)
    ry(np.pi/2, q2)

    # --- Phase 6: Third Entanglement Sequence ---
    rzz(q2, q3, -np.pi/2)
    rx(-np.pi/2, q3)
    ry(-np.pi/2, q2)
    rx(-np.pi, q2)

    # --- Phase 7: Penultimate Entanglement Sequence ---
    rzz(q1, q2, theta)
    rx(np.pi/2, q1)
    rx(np.pi/2, q2)
    ry(-np.pi/2, q1) 
    ry(np.pi/2, q2)

    # --- Phase 8: Final Closing Layer ---
    rzz(q0, q1, np.pi/2)
    rzz(q2, q3, np.pi/2)
    ry(np.pi/2, q1)
    ry(-np.pi/2, q2)
    rx(np.pi/2, q3)

print("Quantum kernels for LABS defined successfully!")

# ============================================================================
# Verification and Drawing
# ============================================================================

@cudaq.kernel
def test_draw_g2(theta: float):
    q = cudaq.qvector(2)
    two_qubit_block(q[0], q[1], theta)

print("--- FIG 3: Two-Qubit Block Architecture ---")
print(cudaq.draw(test_draw_g2, 0.5))

@cudaq.kernel
def test_draw_g4(theta: float):
    q = cudaq.qvector(4)
    four_qubit_block(q[0], q[1],q[2], q[3], theta) # Correctly passing the qvector

print("\n--- FIG 4: Four-Qubit Block Architecture ---")
print(cudaq.draw(test_draw_g4, 0.5))

There are a few additional items we need to consider before completing the final implementation of the entire circuit.  One simplification we can make is that for our problem the $h_i^x$ terms are all 1 and any $h_b^x$ terms are 0, and are only there for generalizations of this model. 

The remaining challenge is derivation of the angles that are used to apply each of the circuits you defined above. These are obtained from two terms $\lambda(t)$ and $\alpha(t)$.  


The $\lambda(t)$ defines an annealing schedule and is generally a Sin function which slowly "turns on" the problem Hamiltonian.  For computing our angles, we need the derivative of $\lambda(t)$.

The $\alpha$ term is a bit trickier and is the solution to a set of differential equations which minimize the distance between $H^1_{CD}(\lambda)$ and $A(\lambda)$.  The result is 

$$\alpha(t) = \frac{-\Gamma_1(t)}{\Gamma_2(t)} $$

Where $\Gamma_1(t)$ and $\Gamma_2(t)$ are defined in equations 16 and 17 of the paper and essentially depend on the structure of the optimization problem.  Curious learners can look at the functions in `labs_utils.py`  to see how these are computed, based on the problem size and specific time step in the Trotter process. 


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 4:</h3>
    <p style="font-size: 16px; color: #333;">
The `compute_theta` function, called in the following cells, requires all indices of the two and four body terms. These will be used again in our main kernel to apply the respective gates.  Use the products in the formula below to finish the function in the cell below.  Save them as `G2` and `G4` where each is a list of lists of indices defining the two and four term interactions. As you are translating an equation to a set of loops, this is a great opportunity to use an AI coding assistant.

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

</div>


## Exercise 4: Enumerating 2-Body and 4-Body LABS Interactions

### Goal

After defining the quantum circuit primitives, the next step is to determine **which qubits interact**
for a given LABS instance of length `N`.

The LABS Hamiltonian can be expressed as a sum of:
- **2-body interaction terms** (G2)
- **4-body interaction terms** (G4)

In this exercise, we programmatically generate all valid index combinations for these interaction
terms and verify that their counts match the theoretical expressions.

---

### What we implemented

We implemented a function `get_interactions(N)` that enumerates:

- **G2 interactions**  
  Pairs of indices `[i, i+k]` corresponding to valid 2-body interaction terms derived from the LABS autocorrelation structure.

- **G4 interactions**  
  Quadruples of indices `[i, i+t, i+k, i+k+t]` corresponding to valid 4-body interaction terms arising from squared autocorrelation contributions.

All indices are converted to **0-based indexing** so they can be used directly in Python and CUDA-Q kernels.

---

### Theoretical validation

To ensure correctness, we independently computed the **theoretical counts** of G2 and G4 terms as functions of `N`:

- G2 count:  
  Sum over `i = 1` to `N-2` of `floor((N - i) / 2)`

- G4 count:  
  Sum over valid `(i, t, k)` ranges derived from the LABS autocorrelation expansion

We implemented `get_theoretical_counts(N)` to compute these values analytically.

---

### Automated verification

We then performed an automated verification across a range of problem sizes using
`automated_verify(N_range)`.

For each `N`, we compare:
- The number of generated G2 interactions vs. the theoretical G2 count
- The number of generated G4 interactions vs. the theoretical G4 count

A test passes only if **both counts match exactly**.

This verification confirms that our interaction enumeration is:
- Complete (no missing terms)
- Non-redundant (no overcounting)
- Consistent with the theoretical LABS Hamiltonian structure

---

### Why this matters

Correct interaction indexing is critical before constructing the quantum Hamiltonian.
Any missing or duplicated interaction would lead to an incorrect energy landscape and
invalidate both classical and quantum optimization results.

This step ensures that subsequent quantum kernels are applied to the **correct interaction structure**
for arbitrary sequence lengths `N`.


In [ ]:
import math

def get_interactions(N):
    G2 = []
    G4 = []
    
    # G2 indices: i from 1 to N-2
    for i in range(1, N - 1): 
        # k from 1 to floor((N-i)/2)
        for k in range(1, (N - i) // 2 + 1):
            # 轉換為 0 索引：(i-1) 和 (i+k-1)
            G2.append([i - 1, i + k - 1])

    # G4 indices: i from 1 to N-3
    for i in range(1, N - 2):
        # t from 1 to floor((N-i-1)/2)
        for t in range(1, (N - i - 1) // 2 + 1):
            # k from t+1 to N-i-t
            for k in range(t + 1, N - i - t + 1):
                # 轉換為 0 索引
                G4.append([i - 1, i + t - 1, i + k - 1, i + k + t - 1])
                
    return G2, G4

def get_theoretical_counts(N):
    # G2 理論值：Sum_{i=1}^{N-2} floor((N-i)/2)
    theo_g2 = sum(math.floor((N - i) / 2) for i in range(1, N - 1))
    
    # G4 理論值：計算 k 的區間長度 (N-i-t) - (t+1) + 1 = N-i-2t
    theo_g4 = 0
    for i in range(1, N - 2):
        for t in range(1, math.floor((N - i - 1) / 2) + 1):
            theo_g4 += (N - i - 2 * t)
            
    return theo_g2, theo_g4

def automated_verify(N_range):
    print(f"{'N':<5} | {'G2 Status':<12} | {'G4 Status':<12} | {'Result'}")
    print("-" * 50)
    for n in N_range:
        g2, g4 = get_interactions(n)
        t2, t4 = get_theoretical_counts(n)
        
        match = (len(g2) == t2) and (len(g4) == t4)
        status = "✅ PASS" if match else "❌ FAIL"
        print(f"{n:<5} | {len(g2):>3}/{t2:<3}      | {len(g4):>3}/{t4:<3}      | {status}")

# 執行驗證
automated_verify(range(5, 15))



<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 5:</h3>
    <p style="font-size: 16px; color: #333;">
You are now ready to construct the entire circuit and run the counteradiabatic optimization procedure. The final kernel needs to apply Equation 15 for a specified total evolution time $T$ and the `n_steps` number of Trotter steps.  It must also take as input, the indices for the two and four body terms and the thetas to be applied each step, as these cannot be computed within a CUDA-Q kernel.

The helper function `compute_theta` computes the theta parameters for you, using a few additional functions in accordance with the equations defined in the paper.
</div>


## Exercise 5: Trotterized Quantum Sampling for LABS (Quantum Seed Generation)

### Goal

After implementing the CUDA-Q operator blocks (Exercise 3) and enumerating the valid interaction indices (Exercise 4),
we constructed a **trotterized quantum circuit** that applies all LABS interaction terms in a controlled sequence.
The purpose of this circuit is to **sample candidate bitstrings** that can be used as high-quality initial solutions
(seeds) for the classical Memetic Tabu Search (MTS).

---

### What we implemented

We implemented a CUDA-Q kernel `trotterized_circuit(N, G2, G4, steps, thetas)` with the following structure:

1. **Initialization**  
   Allocate `N` qubits and apply a layer of Hadamards `h(reg)` to start from a uniform superposition.
   This ensures broad coverage of the search space at the start of the evolution.

2. **Trotterized evolution (steps loop)**  
   For each trotter step, we apply:
   - All **2-qubit interaction blocks** across pairs in `G2`
   - All **4-qubit interaction blocks** across quartets in `G4`

   Each step uses a step-specific parameter `current_theta = thetas[step]`, computed on the host before circuit execution.

3. **Measurement**  
   We measure all qubits with `mz(reg)` and collect sampled bitstrings.

---

### Parameter schedule (theta)

The circuit uses a stepwise parameter schedule `thetas` computed on the CPU using a helper routine
(`utils.compute_theta(...)`) based on the counteradiabatic protocol described in the reference approach.
In our implementation, we pre-compute the full theta list before launching the kernel to keep the kernel logic minimal.

---

### How we use the output

We run `cudaq.sample(...)` to obtain a distribution over bitstrings (and their counts).
The highest-frequency samples are treated as **quantum-generated candidate solutions**.

These samples are then used as **seeds** for the classical MTS optimizer:
- Instead of starting MTS from purely random sequences, we start from quantum-sampled candidates
- This improves early convergence behavior and provides a practical hybrid workflow (quantum sampling → classical refinement)

---

### Notes / limitations

- This implementation uses a small number of trotter steps (`n_steps = 1`) for simplicity and runtime constraints.
  Increasing the number of steps is possible, but increases circuit depth and cost.
- The goal here is not to claim full quantum advantage, but to demonstrate a working and extensible
  quantum-seeded hybrid optimization pipeline for LABS.


In [ ]:
import cudaq
import numpy as np

# ============================================================================
# Trotterized Circuit Kernel
# ============================================================================
@cudaq.kernel
def trotterized_circuit(N: int, G2: list[list[int]], G4: list[list[int]], steps: int, thetas: list[float]):
    reg = cudaq.qvector(N)
    h(reg)
    
    for step in range(steps):
        current_theta = thetas[step]
        
        # Two-qubit interactions
        for pair in G2:
            two_qubit_block(reg[pair[0]], reg[pair[1]], current_theta)
            
        # Four-qubit interactions
        for quartet in G4:
            # Here you pass 4 qubits + 1 float = 5 arguments (MATCHED!)
            four_qubit_block(
                reg[quartet[0]], 
                reg[quartet[1]], 
                reg[quartet[2]], 
                reg[quartet[3]], 
                current_theta
            )

    mz(reg)

# ============================================================================
# Main Execution Logic
# ============================================================================

# Define Simulation Parameters
T = 1.0               # Total evolution time
n_steps = 1           # Number of Trotter steps defined in the exercise
dt = T / n_steps      # Time increment per step
N = 20                # Number of qubits

# Generate interaction indices (verified in Exercise 4)
G2, G4 = get_interactions(N)

# Pre-compute theta parameters using the helper utility
# This is done on the host (CPU) before launching the QPU kernel
thetas = []
for step in range(1, n_steps + 1):
    t = step * dt
    # Compute theta according to the paper's Counteradiabatic protocol
    theta_val = utils.compute_theta(t, dt, T, N, G2, G4)
    thetas.append(theta_val)

# Execute the simulation using the NVIDIA GPU-accelerated backend
# The sample function returns a dictionary of bitstrings and their counts
print(f"Launching simulation for N={N} with {n_steps} Trotter steps...")
result = cudaq.sample(trotterized_circuit, N, G2, G4, n_steps, thetas)

# Output results for population initialization
print("Sampling complete. Top sequences will be used for MTS seeding.")
print(result)

## Generating Quantum Enhanced Results

Recall that the point of this lab is to demonstrate the potential benefits of running a quantum subroutine as a preprocessing step for classical optimization of a challenging problem like LABS. you now have all of the tools you need to try this for yourself.

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 6:</h3>
    <p style="font-size: 16px; color: #333;">
Use your CUDA-Q code to prepare an initial population for your memetic search algorithm and see if you can improve the results relative to a random initial population.  If you are running on a CPU, you will need to run smaller problem instances. The code below sets up the problem

</div>


In [ ]:
# Exercise 6: Quantum-seeded MTS population (fixed for Exercise 5 kernel)
# -------------------------------------------------------------
import inspect
import random
import numpy as np

# Assumes these already exist from previous cells:
# - cudaq imported
# - trotterized_circuit defined as trotterized_circuit(N, G2, G4, steps, thetas)
# - N, G2, G4, n_steps, thetas defined
# - mts, compute_energy, visualize_results, sequence_to_string imported


# ----------------------------
# 1) Sample your CUDA-Q kernel
# ----------------------------
shots = 2000           # increase if you want a cleaner distribution
population_size = 20

def _cudaq_sample_kernel(shots: int):
    """
    Robust sampling wrapper: tries common CUDA-Q sample argument names.
    Returns a dict-like counts object: {bitstring: count}.
    """
    try:
        return cudaq.sample(
            trotterized_circuit,
            N, G2, G4, n_steps, thetas,
            shots_count=shots
        )
    except TypeError:
        # some versions use "shots" instead
        return cudaq.sample(
            trotterized_circuit,
            N, G2, G4, n_steps, thetas,
            shots=shots
        )

counts = _cudaq_sample_kernel(shots)

# Safely extract items (bitstring, count)
try:
    items = list(counts.items())
except Exception:
    items = list(dict(counts).items())

# Sort by frequency so we see the distribution clearly (optional but nice)
items.sort(key=lambda x: x[1], reverse=True)

print(f"Sampling complete: {len(items)} unique bitstrings from {shots} shots.")
print("Top 5 most frequent samples:", items[:5])


# -------------------------------------------
# 2) Bitstring -> ±1 numpy array (LABS format)
# -------------------------------------------
def bitstring_to_pm1_array(bs: str) -> np.ndarray:
    """
    Map measurement string to ±1:
      '0' -> +1
      '1' -> -1
    Returns np.ndarray of shape (N,).
    """
    return np.array([1 if b == "0" else -1 for b in bs], dtype=np.int8)

# Convert unique measured strings into candidates (keep uniqueness for diversity)
candidates = []
for bs, ct in items:
    if len(bs) != N:
        # If backend includes spaces or prefix, you might need to sanitize bs.
        # This is just a safety check.
        continue
    candidates.append(bitstring_to_pm1_array(bs))

print(f"Converted {len(candidates)} candidates of length N={N}.")


# ------------------------------------------------------------
# 3) Rank quantum candidates by classical LABS energy & seed pop
# ------------------------------------------------------------
# Compute energies once so we don't recompute in sorting repeatedly
cand_energies = [(seq, compute_energy(seq)) for seq in candidates]
cand_energies.sort(key=lambda x: x[1])  # low energy first

# Take best unique sequences (diverse seeding)
seed_pop = [seq for (seq, E) in cand_energies[:population_size]]

# If not enough unique quantum samples, fill remainder randomly
while len(seed_pop) < population_size:
    seed_pop.append(np.array([random.choice([-1, 1]) for _ in range(N)], dtype=np.int8))

print("Seed population size:", len(seed_pop))
print("Best few seeded energies:", [compute_energy(s) for s in seed_pop[:5]])


# ----------------------------------------
# 4) Run MTS: random init vs quantum init
# ----------------------------------------
mts_kwargs = dict(
    N=N,
    population_size=population_size,
    max_generations=50,
    p_mut=0.05,
    tabu_iter=100,
    tabu_tenure=7,
    verbose=True
)

# Baseline random
best_seq_rand, best_E_rand, final_pop_rand, hist_rand = mts(**mts_kwargs)

# Seeded run (your mts supports initial_population already)
sig = inspect.signature(mts)
if "initial_population" not in sig.parameters:
    raise RuntimeError("Your mts() does not support initial_population, but your pasted version should.")
best_seq_seed, best_E_seed, final_pop_seed, hist_seed = mts(
    **mts_kwargs,
    initial_population=seed_pop
)


# ---------------------------
# 5) Print + visualize results
# ---------------------------
print("\n==================== COMPARISON ====================")
print(f"Random init  best E: {best_E_rand:>4d} | best seq: {sequence_to_string(best_seq_rand)}")
print(f"Quantum init best E: {best_E_seed:>4d} | best seq: {sequence_to_string(best_seq_seed)}")

fig1 = visualize_results(final_pop_rand, hist_rand, best_seq_rand, best_E_rand,
                         f"Random-seeded MTS (N={N})")
fig2 = visualize_results(final_pop_seed, hist_seed, best_seq_seed, best_E_seed,
                         f"Quantum-seeded MTS (N={N})")

plt.show()
#-+-+----+----++-++--

The results clearly show that a population sampled from CUDA-Q results in an improved distribution and a lower energy final result. This is exactly the goal of quantum enhanced optimization.  To not necessarily solve the problem, but improve the effectiveness of state-of-the-art classical approaches. 

A few major caveats need to be mentioned here. First, We are comparing a quantum generated population to a random sample.  It is quite likely that other classical or quantum heuristics could be used to produce an initial population that might even beat the counteradiabatic method you used, so we cannot make any claims that this is the best. 

Recall that the point of the counteradiabatic approach derived in the paper is that it is more efficient in terms of two-qubit gates relative to QAOA. The benefits of this regime would only truly come into play in a setting (e.g. larger problem instance) where it is too difficult to produce a good initial population with any know classical heuristic, and the counteradiabatic approach is more efficiently run on a QPU compared to alternatives.

We should also note that we are comparing a single sample of each approach.  Maybe the quantum sample got lucky or the randomly generated population was unlucky and a more rigorous comparison would need to repeat the analysis many times to draw any confidently conclusions.  

The authors of the paper discuss all of these considerations, but propose an analysis that is quite interesting related to the scaling of the technique. Rather than run large simulations ourselves, examine their results below. 


<img src="images/quantum_enhanced_optimization_LABS/tabu_search_results.png" width="900">

The authors computed replicate median (median of solving the problem repeated with same setup) time to solutions (excluding time to sample from QPU) for problem sizes $N=27$ to $N=37$. Two interesting conclusions can be drawn from this. First, standard memetic tabu search (MTS) is generally faster than quantum enhanced (QE) MTS.  But there are two promising trends. For larger problems, the QE-MTS experiments occasionally have excellent performance with times to solution much smaller than all of the MTS data points.  These outliers indicate there are certain instances where QE-MTS could provide much faster time-to-solution. 

More importantly, if a line of best fit is calculated using the median of each set of medians, the slope of the QE-MTS line is smaller than the MTS!  This seems to indicate that QE solution of this problem scales $O(1.24^N)$ which is better than the best known classical heuristic ($O(1.34^N)$) and the best known quantum approach (QAOA - $O(1.46^N)$).

For problems of size of $N=47$ or greater, the authors anticipate that QE-MTS could be a promising technique and produce good initial populations that are difficult to obtain classically. 

The study reinforces the potential of hybrid workflows enhanced by quantum data such that a classical routine is still the primary solver, but quantum computers make it much more effective.  Future work can explore improvements to both the quantum and classical sides, such as including GPU accelerated memetic search on the classical side.

## Self-validation To Be Completed for Phase 1

In this section, explain how you verified your results. Did you calculate solutions by hand for small N? Did you create unit tests? Did you cross-reference your Quantum energy values against your Classical MTS results? Did you check known symmetries?

In [ ]:
import numpy as np
import itertools
import random

def to_pm1(bits):
    """Convert iterable of 0/1 bits to ±1 numpy array."""
    return np.array([1 if b == 0 else -1 for b in bits], dtype=np.int8)

def reverse_seq(s):
    return s[::-1].copy()

def flip_all(s):
    return (-s).copy()

# -------------------------
# 1) Manual sanity check N=3
# -------------------------
# Example sequence s = [+1, +1, -1]
# C1 = s1*s2 + s2*s3 = (1*1) + (1*-1) = 0
# C2 = s1*s3 = (1*-1) = -1
# E = C1^2 + C2^2 = 0^2 + (-1)^2 = 1
s = np.array([1, 1, -1], dtype=np.int8)
E_hand = 1
E_code = compute_energy(s)
print("Manual check N=3")
print("Sequence:", s, "Energy(hand):", E_hand, "Energy(code):", E_code)
assert E_code == E_hand, "Manual energy check failed!"
print("✅ Manual check passed.\n")

## LABS Symmetries (Invariances)

The LABS energy is invariant under several transformations, which provide useful
self-validation checks for verifying the correctness of the energy computation.

### 1) Global Sign Flip

Let \(\mathbf{s}' = -\mathbf{s}\), meaning \(s'_i = -s_i\). Then:

$$
\begin{aligned}
C_k(\mathbf{s}')
&= \sum_{i=1}^{N-k} s'_i s'_{i+k} \\
&= \sum_{i=1}^{N-k} (-s_i)(-s_{i+k}) \\
&= \sum_{i=1}^{N-k} s_i s_{i+k} \\
&= C_k(\mathbf{s}).
\end{aligned}
$$

Therefore,

$$
E(\mathbf{s}') = E(\mathbf{s}).
$$

---

### 2) Reversal Symmetry

Let \(\mathbf{s}^{\text{rev}}\) denote the reversed sequence, defined by
\(s^{\text{rev}}_i = s_{N+1-i}\).

Reversal does not change the multiset of pairwise products \(s_i s_{i+k}\) for
each lag \(k\); it only reorders them. Thus:

$$
C_k(\mathbf{s}^{\text{rev}}) = C_k(\mathbf{s}),
$$

which implies:

$$
E(\mathbf{s}^{\text{rev}}) = E(\mathbf{s}).
$$

---

### 3) Combined Reversal and Sign Flip

Since both global sign flip and reversal individually preserve the LABS energy,
their composition also leaves the energy invariant:

$$
E(-\mathbf{s}^{\text{rev}}) = E(\mathbf{s}).
$$

Because these symmetries are guaranteed by the mathematical structure of the LABS
objective, they act as strong invariance tests for validating our
\texttt{compute\_energy} implementation.


In [ ]:
# ----------------------------------------
# 2) Symmetry invariance tests for LABS
# ----------------------------------------
# LABS energy should be invariant under:
# - global sign flip: s -> -s
# - reversal: s -> reverse(s)
# - reversal + sign flip

def symmetry_tests(num_trials=200, N=20):
    for _ in range(num_trials):
        s = np.array([random.choice([-1, 1]) for _ in range(N)], dtype=np.int8)
        E = compute_energy(s)

        s_flip = flip_all(s)
        s_rev  = reverse_seq(s)
        s_both = flip_all(s_rev)

        if compute_energy(s_flip) != E:
            return False, "flip invariance failed"
        if compute_energy(s_rev) != E:
            return False, "reverse invariance failed"
        if compute_energy(s_both) != E:
            return False, "reverse+flip invariance failed"
    return True, "all symmetry tests passed"

ok, msg = symmetry_tests(num_trials=300, N=25)
print("Symmetry tests:", msg)
assert ok, msg
print("✅ Symmetry invariance passed.\n")


## Brute-Force Ground Truth for Small \(N\)

For small sequence lengths \(N\), we can compute the global optimum exactly by
enumerating all \(2^N\) possible sequences using the \(\pm 1\) representation.

$$
\{-1, +1\}^N \quad \text{has size} \quad 2^N.
$$

We compute the LABS energy \(E(\mathbf{s})\) for every
\(\mathbf{s} \in \{-1,+1\}^N\) and take:

$$
E^\star(N) = \min_{\mathbf{s} \in \{-1,+1\}^N} E(\mathbf{s}).
$$

This provides an oracle (ground truth) for \(N \le 10\) (and in practice up to
\(N \le 12\)), allowing us to verify that our Memetic Tabu Search (MTS)
implementation can reach the true optimum when the search space is small enough
to fully enumerate.


In [ ]:
# ---------------------------------------------------
# 3) Brute-force ground truth (small N only)
# ---------------------------------------------------
def brute_force_labs(N):
    """
    Enumerate all 2^N sequences (0/1), convert to ±1, compute energy, return optimum.
    For N <= 10 this is fast enough in a notebook.
    """
    best_E = None
    best_s = None
    for bits in itertools.product([0, 1], repeat=N):
        s = to_pm1(bits)
        E = compute_energy(s)
        if best_E is None or E < best_E:
            best_E = E
            best_s = s.copy()
    return best_s, best_E

for N_small in [4, 5, 6, 7, 8, 9, 10]:
    s_opt, E_opt = brute_force_labs(N_small)
    print(
        f"N={N_small:2d} | "
        f"Brute-force optimal energy (ground truth) = {E_opt:3d} | "
        f"Example optimal sequence = {sequence_to_string(s_opt)}"
    )


## Manual Sanity Check (Example for \(N=3\))

To validate \texttt{compute\_energy}, we compute the LABS energy manually for a
small sequence. Let:

$$
\mathbf{s} = (s_1, s_2, s_3) = (+1, +1, -1).
$$

For \(N=3\), there are two lags: \(k=1\) and \(k=2\).

### Lag \(k=1\)

$$
\begin{aligned}
C_1
&= s_1 s_2 + s_2 s_3 \\
&= (1)(1) + (1)(-1) \\
&= 1 - 1 = 0.
\end{aligned}
$$

### Lag \(k=2\)

$$
\begin{aligned}
C_2
&= s_1 s_3 \\
&= (1)(-1) = -1.
\end{aligned}
$$

Thus, the energy is:

$$
\begin{aligned}
E(\mathbf{s})
&= C_1^2 + C_2^2 \\
&= 0^2 + (-1)^2 = 1.
\end{aligned}
$$

We confirm that our code returns \(E(\mathbf{s}) = 1\).


In [ ]:
# ---------------------------------------------------
# 4) Check MTS reaches the brute-force optimum (small N)
# ---------------------------------------------------
def validate_mts_against_bruteforce(
    N_list=(6, 7, 8, 9, 10),
    population_size=30,
    max_generations=200,
    p_mut=0.05,
    tabu_iter=200,
    tabu_tenure=7,
    trials=3
):
    """
    For each N, compute brute-force optimum and see whether MTS hits it within a few trials.
    """
    results = []
    for N in N_list:
        s_opt, E_opt = brute_force_labs(N)

        hit = False
        best_found = None
        for t in range(trials):
            best_seq, best_E, final_pop, hist = mts(
                N=N,
                population_size=population_size,
                max_generations=max_generations,
                p_mut=p_mut,
                tabu_iter=tabu_iter,
                tabu_tenure=tabu_tenure,
                initial_population=None,
                verbose=False
            )
            best_found = best_E if best_found is None else min(best_found, best_E)
            if best_E == E_opt:
                hit = True
                break

        results.append((N, E_opt, best_found, hit))
        print(
            f"N={N:2d} | "
            f"Brute-force optimal energy = {E_opt:3d} | "
            f"Best MTS energy (over {trials} runs) = {best_found:3d} | "
            f"Optimal reached = {hit}"
        )

    return results

_ = validate_mts_against_bruteforce(
    N_list=(6, 7, 8, 9, 10),
    population_size=30,
    max_generations=250,
    p_mut=0.05,
    tabu_iter=250,
    tabu_tenure=7,
    trials=5
)

## Hybrid Workflow: Quantum-Seeding for MTS

The digitized counterdiabatic circuit prepares a quantum state of the form:

$$
\lvert \psi(\theta) \rangle = U(\theta)\lvert 0 \rangle^{\otimes N},
$$

which induces a measurement distribution over computational basis bitstrings.

$$
p_\theta(x) = \left|\langle x \mid \psi(\theta) \rangle\right|^2,
\quad x \in \{0,1\}^N.
$$

Each measured bitstring \(x\) is converted into a LABS sequence
\(\mathbf{s}(x) \in \{-1,+1\}^N\) using a fixed mapping
(e.g., \(0 \mapsto +1\), \(1 \mapsto -1\)).
These quantum samples provide an **informed initial population** for the
Memetic Tabu Search (MTS).

The classical optimizer then refines these candidates using recombination,
mutation, and tabu search to reach low-energy LABS solutions.

This hybrid approach does not replace classical optimization; rather, it
modifies the **initial sampling distribution** of candidate solutions,
potentially accelerating convergence to optimal or near-optimal sequences.


In [ ]:
import numpy as np
import random
import cudaq
# Assumes these exist already:
# - cudaq imported
# - trotterized_circuit defined as trotterized_circuit(N, G2, G4, steps, thetas)
# - N, G2, G4, n_steps, thetas defined
# - compute_energy defined (LABS energy on ±1)
#
# If your compute_energy expects np.ndarray, we return np.ndarray below.

def bitstring_to_pm1_array(bs: str) -> np.ndarray:
    # 0 -> +1, 1 -> -1
    return np.array([1 if b == "0" else -1 for b in bs], dtype=np.int8)

def sample_quantum_sequences(shots: int):
    """
    Samples the CUDA-Q kernel and returns:
      - seqs: list[np.ndarray] of ±1 sequences
      - counts_items: sorted list of (bitstring, count)
    """
    # Robust shots arg
    try:
        counts = cudaq.sample(trotterized_circuit, N, G2, G4, n_steps, thetas, shots_count=shots)
    except TypeError:
        counts = cudaq.sample(trotterized_circuit, N, G2, G4, n_steps, thetas, shots=shots)

    try:
        items = list(counts.items())
    except Exception:
        items = list(dict(counts).items())

    # Sort by frequency
    items.sort(key=lambda x: x[1], reverse=True)

    # Expand according to counts -> list of samples
    seqs = []
    for bs, ct in items:
        if len(bs) != N:
            # If formatting is weird (spaces/prefix), skip; tell me if you see this.
            continue
        seq = bitstring_to_pm1_array(bs)
        seqs.extend([seq] * ct)

    return seqs, items

def sample_random_sequences(num_samples: int):
    return [np.array([random.choice([-1, 1]) for _ in range(N)], dtype=np.int8) for _ in range(num_samples)]

def energy_stats(seqs):
    energies = np.array([compute_energy(s) for s in seqs], dtype=int)
    return {
        "mean": float(np.mean(energies)),
        "median": float(np.median(energies)),
        "min": int(np.min(energies)),
        "p10": float(np.percentile(energies, 10)),
        "p25": float(np.percentile(energies, 25)),
        "p75": float(np.percentile(energies, 75)),
        "p90": float(np.percentile(energies, 90)),
        "energies": energies
    }

# ----------------------------
# Run the validation
# ----------------------------
shots = 2000  # increase if runtime allows
quantum_seqs, quantum_counts = sample_quantum_sequences(shots=shots)
random_seqs = sample_random_sequences(num_samples=len(quantum_seqs))

print(f"Quantum samples collected: {len(quantum_seqs)} (from {shots} shots)")
print(f"Unique outcomes: {len(quantum_counts)}")
print("Top 10 outcomes (bitstring: count):")
for bs, ct in quantum_counts[:10]:
    print(f"  {bs}: {ct}")

# If everything is ":1", you’re still at effectively one-shot or very flat distribution.
num_repeats = sum(1 for _, ct in quantum_counts if ct > 1)
print(f"Outcomes with count > 1: {num_repeats}")

qs = energy_stats(quantum_seqs)
rs = energy_stats(random_seqs)

print("\nEnergy comparison (lower is better):")
print(f"Quantum  mean={qs['mean']:.2f}, median={qs['median']:.2f}, min={qs['min']}, p10={qs['p10']:.2f}, p25={qs['p25']:.2f}")
print(f"Random   mean={rs['mean']:.2f}, median={rs['median']:.2f}, min={rs['min']}, p10={rs['p10']:.2f}, p25={rs['p25']:.2f}")

# Best-of-K test (important for seeding)
K = 50
q_bestK = int(np.min(qs["energies"][:K])) if len(qs["energies"]) >= K else int(np.min(qs["energies"]))
r_bestK = int(np.min(rs["energies"][:K])) if len(rs["energies"]) >= K else int(np.min(rs["energies"]))
print(f"\nBest-of-{K} (first {K} samples): quantum={q_bestK}, random={r_bestK}")

# ----------------------------
# Quick visualization
# ----------------------------
plt.figure()
plt.hist(qs["energies"], bins=30, alpha=0.6, label="Quantum samples")
plt.hist(rs["energies"], bins=30, alpha=0.6, label="Random samples")
plt.xlabel("LABS Energy")
plt.ylabel("Count")
plt.title(f"Energy distribution: Quantum vs Random (N={N}, shots={shots})")
plt.legend()
plt.show()


## Benchmarking and Performance Evaluation

In this section, we benchmark the performance of the classical Memetic Tabu Search (MTS)
under two different initialization strategies:

1. **Random-seeded MTS**: the population is initialized uniformly at random.
2. **Quantum-seeded MTS**: the initial population is seeded using measurement outcomes
   from the digitized counterdiabatic quantum circuit implemented in Exercise 5.

All other MTS hyperparameters are kept identical. This ensures a fair comparison where
the only difference is the initialization strategy.

We report:
- best final energy achieved,
- convergence behavior over generations,
- and runtime statistics,
averaged over multiple independent trials.


In [ ]:
def bitstring_to_pm1_array(bs: str) -> np.ndarray:
    return np.array([1 if b == "0" else -1 for b in bs], dtype=np.int8)

def build_quantum_seed_population(
    N: int,
    population_size: int,
    shots: int,
    n_steps: int,
    T: float
):
    """
    Build an initial population for MTS using quantum samples.
    """
    # Interaction sets
    G2, G4 = get_interactions(N)

    # Compute thetas
    dt = T / n_steps
    thetas = []
    for step in range(1, n_steps + 1):
        t = step * dt
        thetas.append(utils.compute_theta(t, dt, T, N, G2, G4))

    # Sample the quantum circuit
    try:
        counts = cudaq.sample(
            trotterized_circuit, N, G2, G4, n_steps, thetas, shots_count=shots
        )
    except TypeError:
        counts = cudaq.sample(
            trotterized_circuit, N, G2, G4, n_steps, thetas, shots=shots
        )

    try:
        items = list(counts.items())
    except Exception:
        items = list(dict(counts).items())

    # Convert unique bitstrings to ±1 sequences
    candidates = []
    for bs, ct in items:
        if len(bs) == N:
            candidates.append(bitstring_to_pm1_array(bs))

    # Rank by classical LABS energy
    ranked = [(s, compute_energy(s)) for s in candidates]
    ranked.sort(key=lambda x: x[1])

    seed_pop = [s for s, _ in ranked[:population_size]]

    # Fill remainder randomly if needed
    while len(seed_pop) < population_size:
        seed_pop.append(
            np.array([random.choice([-1, 1]) for _ in range(N)], dtype=np.int8)
        )

    return seed_pop


In [ ]:
def run_one_trial(
    N: int,
    population_size: int,
    max_generations: int,
    p_mut: float,
    tabu_iter: int,
    tabu_tenure: int,
    seed: int,
    quantum: bool,
    shots: int = 2000,
    n_steps: int = 1,
    T: float = 1.0
):
    random.seed(seed)
    np.random.seed(seed)

    initial_population = None
    if quantum:
        initial_population = build_quantum_seed_population(
            N, population_size, shots, n_steps, T
        )

    t0 = time.perf_counter()
    best_seq, best_E, final_pop, history = mts(
        N=N,
        population_size=population_size,
        max_generations=max_generations,
        p_mut=p_mut,
        tabu_iter=tabu_iter,
        tabu_tenure=tabu_tenure,
        initial_population=initial_population,
        verbose=False
    )
    t1 = time.perf_counter()

    return {
        "N": N,
        "quantum_seeded": quantum,
        "best_E": int(best_E),
        "runtime_s": t1 - t0,
        "history": history
    }


In [ ]:
import time
def benchmark(
    N_list=(10, 12, 14, 16),
    trials=5,
    population_size=20,
    max_generations=50,
    p_mut=0.05,
    tabu_iter=100,
    tabu_tenure=7,
    shots=2000,
    n_steps=1,
    T=1.0,
    base_seed=123
):
    rows = []
    histories = []

    for N in N_list:
        for t in range(trials):
            seed = base_seed + 1000 * N + t

            r = run_one_trial(
                N, population_size, max_generations, p_mut,
                tabu_iter, tabu_tenure, seed,
                quantum=False
            )
            q = run_one_trial(
                N, population_size, max_generations, p_mut,
                tabu_iter, tabu_tenure, seed,
                quantum=True, shots=shots, n_steps=n_steps, T=T
            )

            rows.append({k: r[k] for k in ["N", "quantum_seeded", "best_E", "runtime_s"]})
            rows.append({k: q[k] for k in ["N", "quantum_seeded", "best_E", "runtime_s"]})
            histories.extend([r, q])

            print(
                f"N={N}, trial {t+1}/{trials} | "
                f"random E={r['best_E']} | quantum E={q['best_E']}"
            )

    return pd.DataFrame(rows), histories


df_bench, histories = benchmark(
    N_list=(10, 12, 14, 16),
    trials=5
)

df_bench


In [ ]:
summary = (
    df_bench
    .groupby(["N", "quantum_seeded"])
    .agg(
        best_E_mean=("best_E", "mean"),
        best_E_median=("best_E", "median"),
        best_E_min=("best_E", "min"),
        runtime_mean=("runtime_s", "mean")
    )
    .reset_index()
)

summary


In [ ]:
def mean_curve(histories, N, quantum, max_gens):
    curves = [h["history"] for h in histories if h["N"] == N and h["quantum_seeded"] == quantum]
    curves = [
        c[:max_gens+1] if len(c) >= max_gens+1 else c + [c[-1]] * (max_gens+1 - len(c))
        for c in curves
    ]
    return np.mean(np.array(curves), axis=0)

max_gens = 50
for N in sorted(df_bench["N"].unique()):
    plt.figure()
    plt.plot(mean_curve(histories, N, False, max_gens), label="Random initialization")
    plt.plot(mean_curve(histories, N, True, max_gens), label="Quantum initialization")
    plt.xlabel("Generation")
    plt.ylabel("Best energy so far (mean over trials)")
    plt.title(f"Convergence comparison (N={N})")
    plt.legend()
    plt.show()


In [ ]:
# ----------------------------
# Benchmark targets (E_opt)
# ----------------------------
E_OPT = {
    3: 1.0, 4: 2.0, 5: 2.0, 6: 5.0, 7: 3.0, 8: 8.0, 9: 6.0, 10: 13.0,
    11: 5.0, 12: 14.0, 13: 3.0, 14: 22.0, 15: 14.0, 16: 24.0, 17: 21.0, 18: 27.0,
    19: 20.0, 20: 36.0, 21: 22.0, 22: 44.0, 23: 38.0, 24: 42.0, 25: 46.0, 26: 58.0,
    27: 42.0, 28: 62.0, 29: 60.0, 30: 67.0, 31: 72.0, 32: 68.0, 33: 79.0, 34: 87.0,
    35: 86.0, 36: 104.0, 37: 110.0, 38: 118.0, 39: 134.0, 40: 132.0
}

# ----------------------------
# Energy-eval counter wrapper
# ----------------------------
class EnergyCounter:
    def __init__(self, energy_fn):
        self.energy_fn = energy_fn
        self.count = 0

    def reset(self):
        self.count = 0

    def __call__(self, s):
        self.count += 1
        return self.energy_fn(s)

energy_counter = EnergyCounter(compute_energy)



In [ ]:
import time
import numpy as np

def run_mts_to_target(
    N,
    target_E,
    *,
    population_size=20,
    max_generations=200,
    p_mut=0.05,
    tabu_iter=100,
    tabu_tenure=7,
    initial_population=None,
    verbose=False,
):
    """
    Returns:
      solved (bool),
      best_energy (int),
      evals (int)  # number of compute_energy calls (proxy for work)
      wall_time (float)  # seconds
      best_seq (np.ndarray)
    """
    global compute_energy

    # swap in the counting energy function
    original_energy = compute_energy
    compute_energy = energy_counter
    energy_counter.reset()

    t0 = time.time()
    try:
        best_seq, best_E, final_pop, hist = mts(
            N=N,
            population_size=population_size,
            max_generations=max_generations,
            p_mut=p_mut,
            tabu_iter=tabu_iter,
            tabu_tenure=tabu_tenure,
            initial_population=initial_population,
            verbose=verbose
        )

        solved = (best_E <= target_E)
        dt = time.time() - t0
        evals = energy_counter.count

        return solved, best_E, evals, dt, best_seq

    finally:
        # restore original energy function no matter what
        compute_energy = original_energy


In [ ]:
import random

def bitstring_to_pm1(bs: str):
    # '0' -> +1, '1' -> -1
    return np.array([1 if b == "0" else -1 for b in bs], dtype=int)

def quantum_seed_population(
    N, population_size, *,
    shots=2000
):
    """
    Uses your existing CUDA-Q setup:
      - trotterized_circuit
      - G2, G4, n_steps, thetas
    Returns: List[np.ndarray] of length population_size
    """
    # IMPORTANT: set shots explicitly so you don't get weird defaults
    counts = cudaq.sample(trotterized_circuit, N, G2, G4, n_steps, thetas, shots_count=shots)

    # extract (bitstring, count)
    try:
        items = list(counts.items())
    except Exception:
        items = list(dict(counts).items())

    # unique bitstrings → candidate sequences
    candidates = []
    seen = set()
    for bs, ct in items:
        if bs not in seen:
            seen.add(bs)
            candidates.append(bitstring_to_pm1(bs))

    # rank by classical energy (lower is better)
    candidates.sort(key=compute_energy)

    seed = candidates[:population_size]
    while len(seed) < population_size:
        seed.append(np.array([random.choice([-1, 1]) for _ in range(N)], dtype=int))

    return seed


In [ ]:
import matplotlib.pyplot as plt

def benchmark(
    Ns,
    trials=25,
    *,
    population_size=20,
    max_generations=250,
    p_mut=0.05,
    tabu_iter=100,
    tabu_tenure=7,
    quantum_shots=2000,
):
    rows = []

    for N in Ns:
        target_E = E_OPT[N]
        print(f"\n=== N={N}, target E={target_E} ===")

        for r in range(trials):
            # ----- Classical MTS -----
            solved, best_E, evals, dt, _ = run_mts_to_target(
                N, target_E,
                population_size=population_size,
                max_generations=max_generations,
                p_mut=p_mut,
                tabu_iter=tabu_iter,
                tabu_tenure=tabu_tenure,
                initial_population=None,
                verbose=False,
            )
            rows.append(("MTS", N, solved, best_E, evals, dt))

            # ----- Quantum-seeded MTS (QE-MTS) -----
            seed_pop = quantum_seed_population(N, population_size, shots=quantum_shots)
            solved, best_E, evals, dt, _ = run_mts_to_target(
                N, target_E,
                population_size=population_size,
                max_generations=max_generations,
                p_mut=p_mut,
                tabu_iter=tabu_iter,
                tabu_tenure=tabu_tenure,
                initial_population=seed_pop,
                verbose=False,
            )
            rows.append(("QE-MTS", N, solved, best_E, evals, dt))

        print("done.")

    return rows

Ns = list(range(20, 30))   
rows = benchmark(Ns, trials=3, max_generations=200, quantum_shots=1500)


## Benchmark Metric: Time-to-Solution (TTS)

To study scaling behavior, we use a **Time-to-Solution (TTS)** metric consistent
with the benchmarking protocol in the reference paper.

For each problem size \(N\), let \(E^\star(N)\) denote the **known optimal LABS
energy**, obtained either from brute-force enumeration (for small \(N\)) or from
published reference tables. A single run is considered **successful** if it
produces some sequence \(\mathbf{s}\) such that:

$$
E(\mathbf{s}) \le E^\star(N).
$$

We define the **Time-to-Solution** as the number of objective function evaluations
required to reach the target energy **for the first time**. If \(t\) denotes the
index of objective evaluations, then:

$$
\text{TTS}(N) = \min \left\{ t \;:\; E(\mathbf{s}_t) \le E^\star(N) \right\}.
$$

In practice, we measure TTS using the number of calls to
`compute_energy` (objective function evaluations), which is more
hardware-independent than wall-clock time.

We perform \(R\) independent runs for each value of \(N\) and report the **median**
Time-to-Solution:

$$
\text{Median-TTS}(N) =
\operatorname{median}\left(
\text{TTS}_1(N), \ldots, \text{TTS}_R(N)
\right).
$$

We plot Median-TTS versus \(N\) on a logarithmic scale to visualize the expected
exponential scaling behavior.


In [ ]:
import numpy as np

def plot_benchmark(rows, use="evals"):
    # use="evals" or use="time"
    idx = 4 if use == "evals" else 5

    algs = sorted(set(r[0] for r in rows))
    Ns = sorted(set(r[1] for r in rows))

    plt.figure(figsize=(8,5))

    for alg in algs:
        # scatter all trials
        for N in Ns:
            vals = [r[idx] for r in rows if r[0]==alg and r[1]==N and r[2] is True]
            # only plot solved trials (paper’s TTS is defined on success)
            if len(vals):
                plt.scatter([N]*len(vals), vals, alpha=0.35)

        # median per N
        med_x, med_y = [], []
        for N in Ns:
            vals = [r[idx] for r in rows if r[0]==alg and r[1]==N and r[2] is True]
            if len(vals):
                med_x.append(N)
                med_y.append(np.median(vals))
        plt.plot(med_x, med_y, linestyle="--", marker="o", label=alg)

    plt.yscale("log")
    plt.xlabel("Sequence length (N)")
    ylabel = "Median energy evaluations (proxy TTS)" if use=="evals" else "Median wall time (s)"
    plt.ylabel(ylabel)
    plt.title("Benchmark: QE-MTS vs MTS on LABS (solved trials only)")
    plt.legend()
    plt.grid(True, which="both", alpha=0.2)
    plt.show()

plot_benchmark(rows, use="evals")
# plot_benchmark(rows, use="time")  # if you also want seconds


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

def plot_publication_style(rows):
    # --- Settings for "Exactly Like This" ---
    # Colors extracted from the original image
    color_qe = '#2A9D8F'  # Teal for QE-MTS
    color_mts = '#E76F51' # Salmon/Orange for MTS
    
    # Filter data
    # rows format: (Algorithm, N, solved, best_E, evals, dt)
    # We use index 4 (evals) as proxy for TTS, consistent with the 10^6 magnitude in the image
    data_qe = {}
    data_mts = {}
    
    Ns = sorted(list(set(r[1] for r in rows)))
    
    for N in Ns:
        # Get successful runs for QE-MTS
        vals_qe = [r[4] for r in rows if r[0] == 'QE-MTS' and r[1] == N and r[2]]
        if vals_qe: data_qe[N] = vals_qe
        
        # Get successful runs for MTS
        vals_mts = [r[4] for r in rows if r[0] == 'MTS' and r[1] == N and r[2]]
        if vals_mts: data_mts[N] = vals_mts

    fig, ax = plt.subplots(figsize=(10, 6))
    
    # --- Helper to plot Violin + Scatter + Median ---
    def plot_group_style(data_dict, color, label_base, shift):
        x_vals = sorted(data_dict.keys())
        y_arrays = [data_dict[x] for x in x_vals]
        
        # 1. Violin Plot (The shaded distribution background)
        parts = ax.violinplot(y_arrays, positions=x_vals, showmeans=False, showextrema=False, widths=0.6)
        for pc in parts['bodies']:
            pc.set_facecolor(color)
            pc.set_alpha(0.3)
            
        # 2. Scatter Points (The dots)
        for x in x_vals:
            y = data_dict[x]
            # Add small random jitter to x for visibility
            jitter = np.random.normal(0, 0.04, size=len(y))
            ax.scatter(np.array([x]*len(y)) + jitter, y, color=color, s=20, alpha=0.6, edgecolors='none')
            
            # 3. Median Bar (The thick black horizontal line)
            median_val = np.median(y)
            ax.hlines(median_val, x - 0.1, x + 0.1, color='black', linewidth=2, zorder=10)

        # 4. Trend Line (The dashed line)
        # We fit log10(y) = slope * N + intercept
        all_x = []
        all_y = []
        for x, ys in data_dict.items():
            all_x.extend([x] * len(ys))
            all_y.extend(ys)
            
        slope, intercept, _, _, _ = linregress(all_x, np.log10(all_y))
        
        # Calculate scaling factor for the label (e.g. 1.24^N)
        # slope = log10(base), so base = 10^slope
        scaling_base = 10**slope
        
        # Generate line points
        line_x = np.linspace(min(x_vals), max(x_vals), 100)
        line_y = 10**(slope * line_x + intercept)
        
        ax.plot(line_x, line_y, color=color, linestyle='--', linewidth=3, label=f"{label_base} ($\sim {scaling_base:.2f}^N$)")

    # --- Plot Both Groups ---
    if data_qe:
        plot_group_style(data_qe, color_qe, "QE-MTS", 0)
    if data_mts:
        plot_group_style(data_mts, color_mts, "MTS", 0)

    # --- Visual Styling to Match Screenshot ---
    ax.set_yscale('log')
    
    # Ticks and Labels
    ax.set_xticks(Ns)
    ax.set_xticklabels(Ns, rotation=45, fontsize=12)
    ax.set_xlabel('Sequence length ($N$)', fontsize=14)
    ax.set_ylabel('Median Time-to-Solution (TTS)', fontsize=14)
    
    # Legend (Top Left, transparent background)
    legend = ax.legend(loc='upper left', frameon=False, fontsize=14)
    
    # Grid lines (Light horizontal line at 10^6)
    ax.axhline(10**6, color='gray', linestyle='-', linewidth=0.5, alpha=0.3)
    
    # Border thickness
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)

    plt.tight_layout()
    plt.show()

# Run this function using the 'rows' variable from your previous benchmark cells
plot_publication_style(rows)